In [2]:
!pip install mpl_toolkits

ERROR: Could not find a version that satisfies the requirement mpl_toolkits (from versions: none)
ERROR: No matching distribution found for mpl_toolkits


In [10]:
import cv2
import numpy as np
import json
import random

# Parameters
background_size = (1024, 1024)  # Size of the synthetic image
num_clusters = 2  # 2 to 3 clusters
ducks_per_cluster = 500  # 25 ducks in a 50x50 pixel area
cluster_std_dev = 20  # Standard deviation for Gaussian clustering
annotation_file = "synthetic_annotations.json"
duck_image_path = "/Users/aqeelpatel/Downloads/IMG_4406_extracted.png"  # Transparent PNG of a single duck

# Load duck image
def load_duck_image(path):
    duck = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if duck is None:
        raise FileNotFoundError(f"Duck image not found at {path}")
    if duck.shape[2] == 3:  # Add alpha channel if missing
        alpha_channel = np.ones((duck.shape[0], duck.shape[1], 1), dtype=np.uint8) * 255
        duck = np.concatenate((duck, alpha_channel), axis=2)
    return duck

def rotate_duck(duck, angle):
    (h, w) = duck.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_duck = cv2.warpAffine(duck, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))
    return rotated_duck

def place_duck(image, duck, position):
    x, y = position
    h, w, _ = duck.shape
    alpha_s = duck[:, :, 3] / 255.0  # Transparency mask
    alpha_l = 1.0 - alpha_s

    for c in range(3):
        image[y:y+h, x:x+w, c] = (
            alpha_s * duck[:, :, c] + alpha_l * image[y:y+h, x:x+w, c]
        )
    return image

def generate_gaussian_cluster(center, num_ducks, std_dev):
    positions = []
    for _ in range(num_ducks):
        x = int(np.random.normal(center[0], std_dev))
        y = int(np.random.normal(center[1], std_dev))
        positions.append((x, y))
    return positions

def generate_synthetic_image():
    # Create blank background
    background = np.ones((*background_size, 3), dtype=np.uint8) * 255

    # Load duck image
    duck = load_duck_image(duck_image_path)
    duck_height, duck_width, _ = duck.shape

    # Annotations
    annotations = {"annotations": [], "images": [], "categories": [{"id": 1, "name": "eider_duck"}]}

    # Add clusters
    cluster_centers = [
        (random.randint(100, background_size[0] - 100), random.randint(100, background_size[1] - 100))
        for _ in range(num_clusters)
    ]

    annotation_id = 1
    for center in cluster_centers:
        positions = generate_gaussian_cluster(center, ducks_per_cluster, cluster_std_dev)

        for pos in positions:
            x, y = pos

            # Calculate angle for the duck to face the cluster center
            angle = -np.degrees(np.arctan2(center[1] - y, center[0] - x)) + 180

            # Rotate duck
            rotated_duck = rotate_duck(duck, angle)

            # Ensure the duck fits within bounds
            if x + duck_width >= background_size[1] or y + duck_height >= background_size[0] or x < 0 or y < 0:
                continue

            # Place duck
            background = place_duck(background, rotated_duck, (x, y))

            # Add annotation
            annotations["annotations"].append({
                "id": annotation_id,
                "image_id": 1,
                "category_id": 1,
                "bbox": [x, y, duck_width, duck_height],
                "area": duck_width * duck_height,
                "iscrowd": 0,
            })
            annotation_id += 1

    # Add image metadata
    annotations["images"].append({
        "id": 1,
        "file_name": "synthetic_image.jpg",
        "width": background_size[1],
        "height": background_size[0],
    })

    # Save image
    cv2.imwrite("synthetic_image.jpg", background)

    # Save annotations
    with open(annotation_file, "w") as f:
        json.dump(annotations, f, indent=4)

    print("Synthetic image and annotations saved.")

if __name__ == "__main__":
    generate_synthetic_image()


Synthetic image and annotations saved.
